In [18]:
import warnings

import numpy as np
import pandas as pd
import xgboost as xgb

from scipy.sparse import csr_matrix
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.cross_validation import StratifiedKFold, cross_val_score
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA

In [19]:
warnings.filterwarnings("ignore")

In [2]:
train = pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')

In [3]:
features = train.columns[1:-1]
train.insert(1, 'SumZeros', (train[features] == 0).astype(int).sum(axis=1))
test.insert(1, 'SumZeros', (test[features] == 0).astype(int).sum(axis=1))

In [4]:
remove = []
c = train.columns
for i in range(len(c)-1):
    v = train[c[i]].values
    for j in range(i+1, len(c)):
        if np.array_equal(v, train[c[j]].values):
            remove.append(c[j])

In [5]:
train.drop(remove, axis=1, inplace=True)
test.drop(remove, axis=1, inplace=True)

In [6]:
remove = []
for col in train.columns:
    if train[col].std() == 0:
        remove.append(col)

In [7]:
train.drop(remove, axis=1, inplace=True)
test.drop(remove, axis=1, inplace=True)

In [8]:
features = train.columns[1:-1]

In [9]:
pca = PCA(n_components=2)
x_train_projected = pca.fit_transform(normalize(train[features], axis=0))
x_test_projected = pca.transform(normalize(test[features], axis=0))
train.insert(1, 'PCAOne', x_train_projected[:, 0])
train.insert(1, 'PCATwo', x_train_projected[:, 1])
test.insert(1, 'PCAOne', x_test_projected[:, 0])
test.insert(1, 'PCATwo', x_test_projected[:, 1])

In [10]:
tokeep = ['num_var39_0',  # 0.00031104199066874026
          'ind_var13',  # 0.00031104199066874026
          'num_op_var41_comer_ult3',  # 0.00031104199066874026
          'num_var43_recib_ult1',  # 0.00031104199066874026
          'imp_op_var41_comer_ult3',  # 0.00031104199066874026
          'num_var8',  # 0.00031104199066874026
          'num_var42',  # 0.00031104199066874026
          'num_var30',  # 0.00031104199066874026
          'saldo_var8',  # 0.00031104199066874026
          'num_op_var39_efect_ult3',  # 0.00031104199066874026
          'num_op_var39_comer_ult3',  # 0.00031104199066874026
          'num_var41_0',  # 0.0006220839813374805
          'num_op_var39_ult3',  # 0.0006220839813374805
          'saldo_var13',  # 0.0009331259720062209
          'num_var30_0',  # 0.0009331259720062209
          'ind_var37_cte',  # 0.0009331259720062209
          'ind_var39_0',  # 0.001244167962674961
          'num_var5',  # 0.0015552099533437014
          'ind_var10_ult1',  # 0.0015552099533437014
          'num_op_var39_hace2',  # 0.0018662519440124418
          'num_var22_hace2',  # 0.0018662519440124418
          'num_var35',  # 0.0018662519440124418
          'ind_var30',  # 0.0018662519440124418
          'num_med_var22_ult3',  # 0.002177293934681182
          'imp_op_var41_efect_ult1',  # 0.002488335925349922
          'var36',  # 0.0027993779160186624
          'num_med_var45_ult3',  # 0.003110419906687403
          'imp_op_var39_ult1',  # 0.0037325038880248835
          'imp_op_var39_comer_ult3',  # 0.0037325038880248835
          'imp_trans_var37_ult1',  # 0.004043545878693624
          'num_var5_0',  # 0.004043545878693624
          'num_var45_ult1',  # 0.004665629860031105
          'ind_var41_0',  # 0.0052877138413685845
          'imp_op_var41_ult1',  # 0.0052877138413685845
          'num_var8_0',  # 0.005598755832037325
          'imp_op_var41_efect_ult3',  # 0.007153965785381027
          'num_op_var41_ult3',  # 0.007153965785381027
          'num_var22_hace3',  # 0.008087091757387248
          'num_var4',  # 0.008087091757387248
          'imp_op_var39_comer_ult1',  # 0.008398133748055987
          'num_var45_ult3',  # 0.008709175738724729
          'ind_var5',  # 0.009953343701399688
          'imp_op_var39_efect_ult3',  # 0.009953343701399688
          'num_meses_var5_ult3',  # 0.009953343701399688
          'saldo_var42',  # 0.01181959564541213
          'imp_op_var39_efect_ult1',  # 0.013374805598755831
          'PCATwo',  # 0.013996889580093312
          'num_var45_hace2',  # 0.014618973561430793
          'num_var22_ult1',  # 0.017107309486780714
          'saldo_medio_var5_ult1',  # 0.017418351477449457
          'PCAOne',  # 0.018040435458786936
          'saldo_var5',  # 0.0208398133748056
          'ind_var8_0',  # 0.021150855365474338
          'ind_var5_0',  # 0.02177293934681182
          'num_meses_var39_vig_ult3',  # 0.024572317262830483
          'saldo_medio_var5_ult3',  # 0.024883359253499222
          'num_var45_hace3',  # 0.026749611197511663
          'num_var22_ult3',  # 0.03452566096423017
          'saldo_medio_var5_hace3',  # 0.04074650077760498
          'saldo_medio_var5_hace2',  # 0.04292379471228616
          'SumZeros',  # 0.04696734059097978
          'saldo_var30',  # 0.09611197511664074
          'var38',  # 0.1390357698289269
          'var15']  # 0.20964230171073095

In [11]:
features = train.columns[1:-1]
todrop = list(set(tokeep).difference(set(features)))
train.drop(todrop, inplace=True, axis=1)
test.drop(todrop, inplace=True, axis=1)
features = train.columns[1:-1]

In [12]:
split = 10
skf = StratifiedKFold(train.TARGET.values, n_folds=split, shuffle=False, random_state=42)

In [13]:
num_rounds = 350
params = {}
params["objective"] = "binary:logistic"
params["eta"] = 0.03
params["subsample"] = 0.8
params["colsample_bytree"] = 0.7
params["silent"] = 1
params["max_depth"] = 5
params["min_child_weight"] = 1
params["eval_metric"] = "auc"

In [ ]:
blind_roc = 0
test_preds = np.ones(len(test))
for train_index, test_index in skf:
    visibletrain = train.iloc[train_index]
    blindtrain = train.iloc[test_index]
    dvisibletrain = xgb.DMatrix(csr_matrix(visibletrain[features]), visibletrain.TARGET.values, silent=True)
    dblindtrain = xgb.DMatrix(csr_matrix(blindtrain[features]), blindtrain.TARGET.values, silent=True)
    clf = xgb.train(params, dvisibletrain, num_rounds, verbose_eval=False)

    blind_preds = clf.predict(dblindtrain)
    roc = roc_auc_score(blindtrain.TARGET.values, blind_preds)
    blind_roc += roc
    print('Blind ROC:', roc)
    
    dfulltest = xgb.DMatrix(csr_matrix(test[features]), silent=True)
    test_preds *= clf.predict(dfulltest)

In [ ]:
blind_roc / split

In [17]:
params = {}
params["objective"] = "binary:logistic"
params["learning_rate"] = 0.03
params["subsample"] = 0.8
params["colsample_bytree"] = 0.7
params["silent"] = 1
params["max_depth"] = 5
params["min_child_weight"] = 1
params["n_estimators"] = 350
clf = xgb.XGBClassifier(**params)
cross_val_score(clf, train[features].values, train.TARGET.values, scoring="roc_auc", cv=skf, n_jobs=6,
                verbose=1).mean()

c:\python35\lib\site-packages\sklearn\externals\joblib\hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
c:\python35\lib\site-packages\sklearn\externals\joblib\hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
c:\python35\lib\site-packages\sklearn\externals\joblib\hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
c:\python35\lib\site-packages\sklearn

0.84003953276244903

In [27]:
#test_preds = np.power(test_preds, 1./split)
clf.fit(train[features].values, train.TARGET.values)
test_preds = clf.predict_proba(test[features].values)[:, 1]

In [28]:
submission = pd.DataFrame({"ID": test.ID, "TARGET": test_preds})
submission.to_csv("simplexgbtest.csv", index=False)

In [ ]:
# first - 0.83994891870680188, 0.84003953276244903

### Hyperopt

In [ ]:
def hyperopt_train_test(params):
    clf = xgb.XGBClassifier(**params)
    return cross_val_score(clf, train_x, train_y, scoring="roc_auc", cv=cv, n_jobs=common.NCPU, verbose=1).mean()